# Analyse der Solarenergie-Entwicklung in der Schweiz
Dieses Notebook analysiert die Entwicklung der Solarenergie und anderer Energiequellen in der Schweiz der letzten 10 Jahre.

## Daten laden und vorbereiten
Die Daten werden aus einer CSV-Datei geladen, bereinigt und für die Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Datenpfad definieren
file_path_energy_data = '../data/energy_sources/elektrizitaetbilanz_jahreswerte.csv'

# CSV-Datei laden
energy_data = pd.read_csv(file_path_energy_data, delimiter=';')

# Nur relevante Spalten behalten
columns_to_keep = ['Jahr'] + [col for col in energy_data.columns if col.startswith('Erzeugung_')]
energy_data = energy_data[columns_to_keep]

# Neue Spalten für aggregierte Energiequellen erstellen
energy_data['Wasserkraft'] = (energy_data['Erzeugung_laufwerk_GWh'] + energy_data['Erzeugung_speicherwerk_GWh'])
energy_data['Andere_erneuerbare_Energien_GWh'] = (energy_data['Erzeugung_holz_GWh'] + energy_data['Erzeugung_biogas_GWh'] + energy_data['Erzeugung_wind_GWh'])

# Daten filtern: Nur Werte ab 2015
energy_data = energy_data[energy_data['Jahr'] >= 2015]

# Jahre als Kategorien behandeln
energy_data['Jahr'] = energy_data['Jahr'].astype(str)

# Spalten umbenennen für bessere Lesbarkeit
energy_data.rename(columns={
    'Erzeugung_kernkraftwerk_GWh': 'Kernkraft',
    'Erzeugung_andere_total_GWh': 'Thermische Kraft',
    'Erzeugung_photovoltaik_GWh': 'Photovoltaik',
    'Andere_erneuerbare_Energien_GWh': 'Andere erneuerbare Energien'
}, inplace=True)

# Daten für Visualisierung vorbereiten (melt-Operation)
columns_to_melt = ['Wasserkraft', 'Kernkraft', 'Thermische Kraft', 'Photovoltaik', 'Andere erneuerbare Energien']
energy_data_melted = energy_data.melt(
    id_vars=['Jahr'], 
    value_vars=columns_to_melt, 
    var_name='Energiequelle', 
    value_name='Wert'
)

## Flächenchart: Energieproduktion nach Quellen
Das folgende Diagramm zeigt die Entwicklung der Energieproduktion nach Quellen in der Schweiz.

In [ ]:
# Flächenchart erstellen
fig = px.area(
    energy_data_melted,
    x='Jahr',
    y='Wert',
    color='Energiequelle',
    title='Energieproduktion nach Quellen in der Schweiz',
    labels={'Wert': 'Energieproduktion (GWh)', 'Jahr': '', 'Energiequelle': 'Energiequelle'},
    color_discrete_sequence=['#f6c90e', '#3c91e6', '#52de97', '#fa744f', '#8a6ded']  # Konsistente Farben
)

# Layout anpassen
fig.update_layout(
    title=dict(x=0.5),  # Titel zentrieren
    yaxis_title='Energieproduktion (GWh)',
    legend_title='Energiequelle',
    font=dict(size=12),
    margin=dict(t=50, b=50, l=50, r=50),
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5)  # Legende unten zentrieren
)

# Diagramm anzeigen
fig.show()

# Diagramm speichern
fig.write_html('../docs/assets/diagramme/solar_vs_other_energy_sources.html')

## Kuchendiagramm: Vergleich der Energiequellen pro Jahr
Das Kuchendiagramm zeigt die prozentuale Verteilung der Energiequellen für ein ausgewähltes Jahr.

In [ ]:
# Farben für die Energiequellen definieren
farben = {
    "Photovoltaik": "#f6c90e",
    "Kernkraft": "#3c91e6",
    "Wasserkraft": "#52de97",
    "Thermische Kraft": "#fa744f",
    "Andere erneuerbare Energien": "#8a6ded"
}

# Liste der Energiequellen und Jahre aus den Daten extrahieren
quellen = list(farben.keys())
years = energy_data["Jahr"].tolist()

# Werte für das erste Jahr (2015) extrahieren
werte_0 = energy_data[energy_data["Jahr"] == years[0]][quellen].values.flatten()

# Kuchendiagramm für das erste Jahr erstellen
fig = go.Figure()
fig.add_trace(go.Pie(
    labels=quellen,
    values=werte_0,
    name=str(years[0]),
    hole=0.4,  # Donut-Diagramm
    sort=False,  # Reihenfolge der Werte nicht sortieren
    rotation=120,  # Startwinkel des Diagramms
    marker=dict(
        colors=[farben[q] for q in quellen],  # Farben für die Energiequellen
        line=dict(color="#ffffff", width=2)  # Weisser Rand um die Segmente
    ),
    textinfo="percent+label",  # Prozent und Label anzeigen
    hovertemplate="%{label}: %{value:.0f} GWh<br>%{percent:.1%}<extra></extra>",  # Tooltip-Informationen
    insidetextfont=dict(size=14),
    texttemplate="%{label}: %{percent:.1%}"  # Prozentzahlen auf 1 Kommastelle runden
))

# Slider-Schritte für jedes Jahr erstellen
steps = []
for year in years:
    # Werte für das aktuelle Jahr extrahieren
    werte = energy_data[energy_data["Jahr"] == year][quellen].values.flatten()
    
    # Schritt für den Slider definieren
    step = dict(
        method="restyle",  # Methode zum Aktualisieren des Diagramms
        args=[{
            "values": [werte],  # Neue Werte für das Diagramm
            "marker": [dict(colors=[farben[q] for q in quellen])],  # Farben beibehalten
            "sort": [False],  # Sortierung beibehalten
            "rotation": [120]  # Rotation beibehalten
        }],
        label=str(year)  # Jahr als Label für den Slider
    )
    steps.append(step)

# Slider-Layout definieren
sliders = [dict(
    active=0,  # Standardmässig das erste Jahr aktivieren
    currentvalue={"prefix": "Jahr: ", "font": {"size": 16}},  # Aktuelles Jahr anzeigen
    pad={"t": 50},  # Abstand zum Diagramm
    steps=steps  # Schritte hinzufügen
)]

# Layout des Diagramms aktualisieren
fig.update_layout(
    title=dict(
        text="Stromerzeugung Schweiz – Prozentuale Verteilung nach Energiequellen (GWh)",
        x=0.5,  # Titel zentrieren
        font=dict(size=20)  # Schriftgrösse des Titels
    ),
    sliders=sliders,  # Slider hinzufügen
    font=dict(family="Arial", size=14),  # Schriftart und -grösse
    margin=dict(t=100, b=50, l=50, r=50),  # Ränder des Diagramms
    showlegend=False
)

# Diagramm als HTML-Datei speichern
fig.write_html("../docs/assets/diagramme/stromproduktion_kuchendiagramm_slider.html")

# Diagramm anzeigen
fig.show()

## Interaktive Treemap: Energiequellen nach Jahr
Die folgende interaktive Treemap visualisiert die Verteilung der Energiequellen in der Schweiz für jedes Jahr. Über den Slider kann das Jahr ausgewählt werden.

In [ ]:
import plotly.graph_objects as go

# Für die Treemap: Daten für jedes Jahr vorbereiten
frames = []
for year in years:
    values = energy_data[energy_data["Jahr"] == year][quellen].values.flatten()
    frames.append(dict(
        data=[go.Treemap(
            labels=quellen,
            parents=[""] * len(quellen),
            values=values,
            marker=dict(colors=[farben[q] for q in quellen]),
            textinfo="label+percent entry+value",
            texttemplate="%{label}<br>%{percentEntry:.1%}<br>%{value:.0f} GWh",  # Wert mit GWh anzeigen
            hovertemplate="%{label}: %{value:.0f} GWh<br>%{percentEntry:.1%}<extra></extra>"
        )],
        name=str(year)
    ))

# Initial-Treemap für das erste Jahr
fig_treemap = go.Figure(
    data=[go.Treemap(
        labels=quellen,
        parents=[""] * len(quellen),
        values=energy_data[energy_data["Jahr"] == years[0]][quellen].values.flatten(),
        marker=dict(colors=[farben[q] for q in quellen]),
        textinfo="label+percent entry+value",
        texttemplate="%{label}<br>%{percentEntry:.1%}<br>%{value:.0f} GWh",  # Wert mit GWh anzeigen
        hovertemplate="%{label}: %{value:.0f} GWh<br>%{percentEntry:.1%}<extra></extra>"
    )],
    frames=frames
)

# Slider für die Jahre
sliders_dict = {
    "active": 0,
    "steps": [
        {
            "args": [[str(year)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
            "label": str(year),
            "method": "animate"
        }
        for year in years
    ],
    "x": 0.5,
    "xanchor": "center",
    "y": -0.1,
    "yanchor": "top",
    "currentvalue": {"prefix": "Jahr: ", "font": {"size": 16}},
    "pad": {"t": 50}
}

fig_treemap.update_layout(
    title=dict(
        text="Treemap der Stromerzeugung nach Energiequellen (GWh)",
        x=0.5,
        font=dict(size=20)
    ),
    margin=dict(t=100, b=50, l=50, r=50),
    sliders=[sliders_dict],
    font=dict(family="Arial", size=14),
    updatemenus=[{
        "type": "buttons",
        "showactive": False,
        "buttons": [
            {
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 800, "redraw": True}, "fromcurrent": True}]
            },
            {
                "label": "Pause",
                "method": "animate",
                "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}]
            }
        ],
        "direction": "left",
        "x": 0.5,
        "xanchor": "center",
        "y": -0.18,
        "yanchor": "top"
    }]
)

# Diagramm als HTML-Datei speichern
fig_treemap.write_html("../docs/assets/diagramme/stromproduktion_treemap_slider.html")

fig_treemap.show()